In [0]:
data = [
    ("101","Alice","2023-11-01", 1000),
    ("102", "Bob", "2023-11-01", 2000),
    ]

col = ["ordre_id","cutomer", "order_date", "amount"]

df = spark.createDataFrame(data, col)
display(df)

In [0]:
df.write.format('delta').mode('overwrite').save('/dbfs/FileStore/bronze_sales')

In [0]:
display(dbutils.fs.ls("tmp/bronze_sales/"))

## Build Medallion Architecutre(Practical)

In [0]:
#Bronze layer - Ingest from RAW CSV

df_raw = spark.read.option("header", True).csv("/databricks-datasets/retail-org/customers/")
#df_raw.write.format("delta").mode("overwrite").save("/tmp/lakehouse/bronze/customers")
df_raw.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze_customers")

display(df_raw)

In [0]:
# Silver Layer: Clean and standard data

# Bronze Layer (assuming already created as a table)
df_bronze = spark.read.table("bronze_customers")

from pyspark.sql.functions import col, lower, trim

# Clean & Standardize Data
df_silver = df_bronze.withColumn("customer_name", lower(trim(col("customer_name")))) \
    .dropna(subset=["customer_id", "customer_name"])

# Save as Silver Layer (Managed Delta Table)
df_silver.write.format("delta").mode("overwrite").saveAsTable("silver_customers")


In [0]:
display(df_silver)

In [0]:
#Gold Layer: Aggregated Insights
# Read Silver as a managed Delta table
df_silver = spark.read.table("silver_customers")

# Gold Aggregation
df_gold = df_silver.groupBy("state").count()

# Save as Gold Layer
df_gold.write.format("delta").mode("overwrite").saveAsTable("customer_summary")

In [0]:
%sql 
select * from customer_summary

Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

In [0]:
%sql
describe history customer_summary